# MLP Multi-Class Class Classification

In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm


### Activation Functions and Derivatives

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

### MLP Classifier

In [3]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

def softmax(x):
    exp_x = torch.exp(x - torch.max(x, dim=1, keepdim=True).values)
    return exp_x / torch.sum(exp_x, dim=1, keepdim=True)

class MLPClassifier:
    def __init__(self, input_size, hidden_layers, output_size, activation='relu', learning_rate=0.01):
        self.learning_rate = learning_rate
        self.activation = activation
        self.output_size = output_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.layers = [input_size] + hidden_layers + [output_size]
        self.weights = []
        self.biases = []

        for i in range(len(self.layers) - 1):
            self.weights.append(torch.randn(self.layers[i], self.layers[i + 1], device=self.device) * 0.01)
            self.biases.append(torch.zeros((1, self.layers[i + 1]), device=self.device))

    def _activate(self, x):
        if self.activation == 'sigmoid':
            return torch.sigmoid(x)
        elif self.activation == 'tanh':
            return torch.tanh(x)
        elif self.activation == 'relu':
            return F.relu(x)
        else:
            raise ValueError("Invalid activation function")

    def forward_propagation(self, X):
        activations = [X]
        zs = []

        for i in range(len(self.weights) - 1):  # Hidden layers
            z = torch.matmul(activations[-1], self.weights[i]) + self.biases[i]
            zs.append(z)
            activations.append(self._activate(z))

        z = torch.matmul(activations[-1], self.weights[-1]) + self.biases[-1]
        zs.append(z)
        activations.append(softmax(z))  # softmax for multi-class classification
        
        return activations, zs

    def compute_loss(self, y_true, y_pred):
        return -torch.mean(torch.sum(y_true * torch.log(y_pred + 1e-9), dim=1))

    def backward_propagation(self, X, y, activations, zs):
        m = X.shape[0]
        deltas = [activations[-1] - y]

        for i in range(len(self.weights) - 1, 0, -1):
            dz = torch.matmul(deltas[0], self.weights[i].T)
            da = torch.where(zs[i - 1] > 0, 1, 0) if self.activation == 'relu' else activations[i] * (1 - activations[i])
            delta = dz * da
            deltas.insert(0, delta)

        dW = [torch.matmul(activations[i].T, deltas[i]) / m for i in range(len(self.weights))]
        dB = [torch.sum(deltas[i], dim=0, keepdim=True) / m for i in range(len(self.weights))]

        return dW, dB

    def update_weights(self, dW, dB):
        for i in range(len(self.weights)):
            self.weights[i] -= self.learning_rate * dW[i]
            self.biases[i] -= self.learning_rate * dB[i]

    def train(self, X, y, X_test, y_test, epochs=1000, method='batch', batch_size=32):
        if method not in ['batch', 'sgd', 'mini-batch']:
            raise ValueError("Invalid method! Choose 'batch', 'sgd', or 'mini-batch'.")
        
        X = torch.tensor(X, dtype=torch.float32, device=self.device)
        y = torch.tensor(y, dtype=torch.float32, device=self.device)
        X_test = torch.tensor(X_test, dtype=torch.float32, device=self.device)
        y_test = torch.tensor(y_test, dtype=torch.float32, device=self.device)

        for epoch in tqdm(range(epochs), desc="Training"):
            if method == 'batch':
                activations, zs = self.forward_propagation(X)
                loss = self.compute_loss(y, activations[-1])
                dW, dB = self.backward_propagation(X, y, activations, zs)
                self.update_weights(dW, dB)

            elif method == 'mini-batch':
                indices = torch.randperm(X.shape[0])
                for i in range(0, X.shape[0], batch_size):
                    batch_indices = indices[i:i + batch_size]
                    X_batch, y_batch = X[batch_indices], y[batch_indices]
                    activations, zs = self.forward_propagation(X_batch)
                    dW, dB = self.backward_propagation(X_batch, y_batch, activations, zs)
                    self.update_weights(dW, dB)

                loss = self.compute_loss(y, self.forward_propagation(X)[0][-1])

            elif method == 'sgd':  # Stochastic GD
                indices = torch.randperm(X.shape[0])
                for i in tqdm(range(X.shape[0]), desc="Samples"):
                    X_sample = X[indices[i]].unsqueeze(0)
                    y_sample = y[indices[i]].unsqueeze(0)
                    activations, zs = self.forward_propagation(X_sample)
                    dW, dB = self.backward_propagation(X_sample, y_sample, activations, zs)
                    self.update_weights(dW, dB)

                loss = self.compute_loss(y, self.forward_propagation(X)[0][-1])

            test_activations, _ = self.forward_propagation(X_test)
            test_loss = self.compute_loss(y_test, test_activations[-1])

            if epoch % 1000 == 0:
                print(f"Epoch {epoch}: Training Loss = {loss.item():.4f}, Testing Loss = {test_loss.item():.4f}")

    def predict(self, X):
        X = torch.tensor(X, dtype=torch.float32, device=self.device)
        activations, _ = self.forward_propagation(X)
        return torch.argmax(activations[-1], dim=1).cpu().numpy()

    def save_model(self, path="mlp_model.pth"):
        model_state = {
            "weights": self.weights,
            "biases": self.biases
        }
        torch.save(model_state, path)
        print(f"Model saved to {path}")

    def load_model(self, path="mlp_model.pth"):
        model_state = torch.load(path, map_location=self.device)
        self.weights = model_state["weights"]
        self.biases = model_state["biases"]
        print(f"Model loaded from {path}")

    def compute_accuracy(self, X, y):
        X = torch.tensor(X, dtype=torch.float32, device=self.device)
        y = torch.tensor(y, dtype=torch.float32, device=self.device)

        predictions = self.predict(X)
        y_true = torch.argmax(y, dim=1).cpu().numpy()

        accuracy = np.mean(predictions == y_true) * 100
        return accuracy



### Loading the Data

In [4]:
def load_data(csv_path, image_folder):
    data = pd.read_csv(csv_path)
    images = []
    labels = data['symbol_id'].values

    for path in tqdm(data['path'], desc=f"Loading images from {csv_path}"):
        img = cv2.imread(os.path.join(image_folder, os.path.basename(path)), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (32, 32)).flatten()
        images.append(img / 255.0)

    return np.array(images), labels

In [5]:
# One-Hot Encoding
def preprocess_labels(labels):
    encoder = LabelEncoder()
    labels_encoded = encoder.fit_transform(labels)
    one_hot_encoder = OneHotEncoder(sparse_output=False)
    labels_one_hot = one_hot_encoder.fit_transform(labels_encoded.reshape(-1, 1))
    return labels_one_hot, encoder

train_csv = 'classification-task/fold-1/train.csv'
test_csv = 'classification-task/fold-1/test.csv'
image_folder = 'images'

X_train, y_train = load_data(train_csv, image_folder)
X_test, y_test = load_data(test_csv, image_folder)
y_train, encoder = preprocess_labels(y_train)
y_test, _ = preprocess_labels(y_test)

Loading images from classification-task/fold-1/train.csv: 100%|██████████| 151241/151241 [00:06<00:00, 23529.98it/s]
Loading images from classification-task/fold-1/test.csv: 100%|██████████| 16992/16992 [00:00<00:00, 26545.06it/s]


In [6]:
mlp = MLPClassifier(input_size=1024, hidden_layers=[64, 32], output_size=y_train.shape[1], activation='relu', learning_rate=0.1)
mlp.train(X_train, y_train, X_test, y_test, epochs=10000, method='batch')

mlp.save_model("trained_mlp_batch.pth")

predictions = mlp.predict(X_test)

Training:   0%|          | 15/10000 [00:00<02:51, 58.27it/s]

Epoch 0: Training Loss = 5.9108, Testing Loss = 5.9104


Training:  10%|█         | 1015/10000 [01:14<06:46, 22.10it/s]

Epoch 1000: Training Loss = 5.3607, Testing Loss = 5.3724


Training:  20%|██        | 2015/10000 [02:10<06:03, 21.96it/s]

...

Epoch 8000: Training Loss = 1.4199, Testing Loss = 1.4146


Training:  90%|█████████ | 9015/10000 [08:44<00:44, 21.90it/s]

Epoch 9000: Training Loss = 1.3436, Testing Loss = 1.3434


Training: 100%|██████████| 10000/10000 [09:40<00:00, 17.23it/s]


Model saved to trained_mlp_batch.pth
